# Database Initiation and Further Statistical Analysis
## Initiate and Import Required Dependencies, Packages, APIs, and Project-Specific Methods

In [6]:
# Miscelaneous
import matplotlib.mlab as mlab
import numpy as np
from pprint import pprint
import json

# Utilities
import pandas as pd
import networkx as nx
import re

# Plotting
import matplotlib.pyplot as plt

# APIs
import pymongo
import tweepy
import googlemaps

# Analysis
import math
from scipy import stats
from scipy.stats import norm

# Get API Keys
file_path = 'keys.txt'
with open(file_path, 'r') as file:
    keys = file.readlines()

In [7]:
import mongosna

In [8]:
google = keys[0]
mongo = "mongodb://localhost:27017/"
twitter = keys[1]

gmaps_client = mongosna.gmaps_client = googlemaps.Client(key=google) # Initiate connection to GoogleMaps API
db_client = mongosna.db_client = pymongo.MongoClient(mongo) # Initiate connection to MongoDB
tweepy_client = mongosna.tweepy_client = tweepy.Client(bearer_token=twitter, wait_on_rate_limit=True, return_type=[dict]) # Initiate tweepy client

In [ ]:
pFollower = mongosna.pFollower = mongosna.db_client.sna_database.primaryFollowers
sFollower = mongosna.sFollower = mongosna.db_client.sna_database.secondaryFollowers
sFollowing = mongosna.sFollowing = mongosna.db_client.sna_database.secondaryFollowing
combined = mongosna.combined = mongosna.db_client.sna_database.combined
sna_database = mongosna.sna_database = mongosna.db_client.sna_database
df_pFollower = mongosna.df_pFollower = pd.DataFrame(list(mongosna.pFollower.find()))
df_sFollower = mongosna.df_sFollower = pd.DataFrame(list(mongosna.sFollower.find().sort("_id",1)))
df_sFollowing = mongosna.df_sFollowing = pd.DataFrame(list(mongosna.sFollowing.find().sort("_id",1)))

## Primary Followers
### Create and Insert User Information into Database
Using the `pymongo` and `tweepy` APIs we will be fetching a more complete dataset of the followers of the Missoula County Twitter account. This will allow us to do further analysis on:<br>
* Followers, following, and tweet count,
* Location,
* Sentiment based on account description,
* Verified accounts<br>
Now that we have a more complete set of user information, we can include that data in our `nodes.csv` file for when we analyze with Gephi.

In [ ]:
fields = ["description","location","entities","verified","public_metrics","profile_image_url"]
_id = 0
users = list(df_pFollower['Id'])

for user in users:
    response = tweepy_client.get_user(id=user, user_fields=fields)

    userDoc = {
        '_id':_id,
        'username':response.data.username,
        'userID':response.data.id,
        'description':response.data.description,
        'location':response.data.location,
        'followers':response.data.public_metrics['followers_count'],
        'following':response.data.public_metrics['following_count'],
        'tweet_count':response.data.public_metrics['tweet_count'],
        'verified':response.data.verified,
        'profile_image_url':response.data.profile_image_url
    }

    pFollower.insert_one(userDoc)
    _id += 1


### Filter Users Based on Location
Since we are connected to a MongoDB Database, we are able to query the documents for users with their locations set to Montana, Missoula, or any other variation on the name "Montana".<br>
This variations include:
* Big Sky Country,
* Last Best Place,
* Montucky,
* and Mont

In [ ]:
query = {"location": {"$regex": ".*mt.*|.*mont.*|.*missoula.*|.*last best place.*|.*big sky.*","$options":"i"}}
mt_users = pd.DataFrame(list(pFollower.find(query)))
mt_users['description'] = mt_users.replace({r'\s+$': '', r'^\s+': ''}, regex=True, inplace=True)
# mt_users.to_csv("pFollowers/pFollower_MT_info.csv",index=False)
mt_users.head()

In [ ]:
# Add 'in_montana' column to DataFrame
mt_ids = list(mt_users['userID'])
user_id = list(df_pFollower['userID'])
bool_list = []

for id in user_id:
    if id in mt_ids:
        bool_list.append(True)
    else:
        bool_list.append(False)
df_pFollower.insert(loc=10,column='in_montana',value=bool_list)

# Add 'in_montana' to database

in_montana = list(df_pFollower['in_montana'])
iter = 0
for id in user_id:
    query = { 'userID' : id }
    value = { '$set' : { 'in_montana' : in_montana[iter]}}
    iter += 1
    pFollower.update_one(query,value)


Now that we have updated and inserted new data into the pandas DataFrame, we can now updated our `nodes.csv` file, allowing us more granular control over the visualizations down the line.<br>
We remove the `'_id'` column as it is unnecessary for the visualizations.

In [ ]:
df_pFollower.drop(columns=["_id"],inplace=True)
df_pFollower.to_csv("pFollowers/nodes.csv",index=False)

### Geocoding
In order to visualize where the users are in relation to the Missoula County, we need to geocode the location data that we have.

In [ ]:
query = {'location':{'$ne':None}}
geoData = pd.DataFrame(list(pFollower.find(query)))
# Add Geocoded data to database
geoData['lat'] = geoData['location'].apply(mongosna.geocode_lat)
geoData['lng'] = geoData['location'].apply(mongosna.geocode_lng)

lat = list(geoData['lat'])
lng = list(geoData['lng'])
geo_ids = list(geoData['_id'])
user_ids = list(df_pFollower['_id'])

mongosna.add_geo(lat,lng,geo_ids,user_ids,pFollower)

TODO: idk something cool

### Analysis
Now that we have information like followers, following, and tweet count, we can get a better picture as to how this group of users statistically looks. Here we are using the built in `mean()`, `median()`, `mode()`, `min()`, `max()`, `std()`, and `var()` methods from `pandas`.<br>
<br>
The following analysis is on the whole dataset. Following this we will analyze the users in Montana.

In [ ]:
mongosna.stats(df=df_pFollower,output_path='pFollowers/stats.json')

In [ ]:
mongosna.makeStatsGraph(output_path='pFollowers/figs/followers.png',
               data_list=list(df_pFollower['followers']),
               xlabel='Followers',
               title=r'$\mathrm{Histogram\ of\ Followers:}\ \mu=%.3f,\ \sigma=%.3f$')

In [ ]:
mongosna.makeStatsGraph(output_path='pFollowers/figs/following.png',
               data_list=list(df_pFollower['following']),
               xlabel='Following',
               title=r'$\mathrm{Histogram\ of\ Following:}\ \mu=%.3f,\ \sigma=%.3f$')

In [ ]:
mongosna.makeStatsGraph(output_path='pFollowers/figs/tweetcount.png',
               data_list=list(df_pFollower['tweet_count']),
               xlabel='Tweets',
               title=r'$\mathrm{Histogram\ of\ Tweet Count:}\ \mu=%.3f,\ \sigma=%.3f$')

#### Analysis of Montana Users

In [18]:
query = {"in_montana": True}
mt_users = pd.DataFrame(list(pFollower.find(query)))
mongosna.stats(mt_users,output_path='pFollowers/stats/mt_stats.json')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['followers']),
               xlabel='Followers',
               title=r'$\mathrm{Histogram\ of\ Followers:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'pFollowers/figs/mt_users/followers_mt.png')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['following']),
               xlabel='Following',
               title=r'$\mathrm{Histogram\ of\ Following:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'pFollowers/figs/mt_users/following_mt.png')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['tweet_count']),
               xlabel='Tweets',
               title=r'$\mathrm{Histogram\ of\ Tweet Count:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'pFollowers/figs/mt_users/tweetcount_mt.png')

## Secondary Followers
### Data Cleaning and Preparation
Based on the community detection algorithm ran previously, we are able to put those values into the database.

In [ ]:
# Add already existing users to secondaryFollowers Collection
df_sFollowerNodes = pd.read_csv('sFollowers/nodes.csv')
sFollower = sna_database["secondaryFollowers"]
list_pFollowers = list(df_pFollower['Id'])
list_sFollowers = list(df_sFollowerNodes['id'])
temp = [user for user in list_pFollowers if user in list_sFollowers]

for i in range(len(temp)):
    userDoc = pFollower.find_one({"userID":temp[i]})
    sFollower.insert_one(userDoc)

In [ ]:
# Add the new users to the secondaryFollowers Collection
fields = ["description","location","entities","verified","public_metrics","profile_image_url"]
_id = 0
temp = [user for user in list_sFollowers if user not in list_pFollowers]
for i in range(len(temp)):
    user = temp[i]
    query = {"_id":_id}
    test = sFollower.find_one(query)
    if type(test) != dict:
        try:
            response = api.get_user(id=user, user_fields=fields)

            userDoc = {
                '_id':_id,
                'username':response.data.username,
                'userID':response.data.id,
                'description':response.data.description,
                'location':response.data.location,
                'followers':response.data.public_metrics['followers_count'],
                'following':response.data.public_metrics['following_count'],
                'tweet_count':response.data.public_metrics['tweet_count'],
                'verified':response.data.verified,
                'profile_image_url':response.data.profile_image_url
            }

            sFollower.insert_one(userDoc)
        except:
            print(str(user) + " no longer exists")
    else:
        pass
    _id += 1

In [ ]:
# Add geocoded Data to the database
query = {'location': {'$ne': None}}
geoData = pd.DataFrame(list(sFollower.find(query).sort("_id",1)))

geoData['lat'] = geoData['location'].apply(mongosna.geocode_lat)
geoData['lng'] = geoData['location'].apply(mongosna.geocode_lng)

lat = list(geoData['lat'])
lng = list(geoData['lng'])
geo_ids = list(geoData['_id'])
iter = 0

for id in geo_ids:
    query = { '_id' : id }
    value = { '$set' : { 'lat' : lat[iter], 'lng': lng[iter]}}
    iter += 1
    sFollower.update_one(query,value)

In [ ]:
# Check if user is in montana
query = {"location": {"$regex": ".*mt.*|.*mont.*|.*missoula.*|.*last best place.*|.*big sky.*","$options":"i"}}
mt_users = pd.DataFrame(list(sFollower.find(query)))
mt_users['description'] = mt_users.replace({r'\s+$': '', r'^\s+': ''}, regex=True, inplace=True)

# Add 'in_montana' column to DataFrame
df_sFollower.drop(columns="in_montana",inplace=True)
mt_ids = list(mt_users['userID'])
user_id = list(df_sFollower['userID'])
bool_list = []

for id in user_id:
    if id in mt_ids:
        bool_list.append(True)
    else:
        bool_list.append(False)
df_sFollower.insert(loc=10,column='in_montana',value=bool_list)

# Add 'in_montana' to database

in_montana = list(df_sFollower['in_montana'])
iter = 0
for id in user_id:
    query = { 'userID' : id }
    value = { '$set' : { 'in_montana' : in_montana[iter]}}
    iter += 1
    sFollower.update_one(query,value)


### Remove Users from the Edges Table if they are not in the Database

In [ ]:
df_sFollowerEdges = pd.read_csv("sFollowers/edges.csv")
sFollower_ids = list(df_sFollower['userID'])

for i in range(len(sFollower_ids)):
    df_sFollowerEdges = df_sFollowerEdges.loc[df_sFollowerEdges['target'] != sFollower_ids[i]]

df_sFollowerEdges.to_csv('sFollowers/edges.csv',index=False)
df_sFollowerEdges.info()

### Analysis

In [ ]:
mongosna.stats(df=df_sFollower,output_path='sFollowers/stats.json')

In [ ]:
mongosna.makeStatsGraph(list(df_sFollower['followers']),
               xlabel='Followers',
               title=r'$\mathrm{Histogram\ of\ Followers:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'sFollowers/figs/followers.png')

In [ ]:
mongosna.makeStatsGraph(list(df_sFollower['following']),
               xlabel='Following',
               title=r'$\mathrm{Histogram\ of\ Following:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'sFollowers/figs/following.png')

In [ ]:
mongosna.makeStatsGraph(list(df_sFollower['tweet_count']),
               xlabel='Following',
               title=r'$\mathrm{Histogram\ of\ Following:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path = 'sFollowers/figs/following.png')

#### Analysis of Montana Users

In [26]:
query = {"in_montana": True}
mt_users = pd.DataFrame(list(sFollower.find(query)))
mongosna.stats(df=df_sFollower,output_path='sFollowers/mt_stats.json')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['followers']),
               xlabel='Followers',
               title=r'$\mathrm{Histogram\ of\ Followers:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path='sFollowers/figs/mt_users/followers_mt.png')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['following']),
               xlabel='Following',
               title=r'$\mathrm{Histogram\ of\ Following:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path='sFollowers/figs/mt_users/following_mt.png')

In [ ]:
mongosna.makeStatsGraph(data_list=list(mt_users['tweet_count']),
               xlabel='Tweets',
               title=r'$\mathrm{Histogram\ of\ Tweet Count:}\ \mu=%.3f,\ \sigma=%.3f$',
               output_path='sFollowers/figs/mt_users/tweetcount_mt.png')

## Secondary Following

In [16]:
temp = pd.read_csv('sFollowing/nodes.csv')
temp.drop(columns=['Unnamed: 0'],inplace=True)
temp.to_csv('sFollowing/nodes.csv',index=False)

In [17]:
temp = pd.read_csv('sFollowing/edges.csv')
temp.drop(columns=['lre'],inplace=True)
temp.to_csv('sFollowing/edges.csv',index=False)

In [5]:
# Add already existing users to secondaryFollowers Collection

df_sFollowingNodes = pd.read_csv('sFollowing/nodes.csv')
df_sFollowingNodes.drop(columns=["degree"],inplace=True)

list_pFollowers = list(df_pFollower['userID'])
list_sFollowing = list(df_sFollowingNodes['id'])
temp = [user for user in list_pFollowers if user in list_sFollowing]

for i in range(len(temp)):
    userDoc = pFollower.find_one({"userID":temp[i]})
    sFollowing.insert_one(userDoc)

In [ ]:
# Add the new users to the secondaryFollowers Collection
fields = ["description","location","entities","verified","public_metrics","profile_image_url"]
df_sFollowingEdges = pd.read_csv("sFollowing/edges.csv")

temp = [user for user in list_sFollowing if user not in list_pFollowers]

_id = 0
total_calls = math.ceil(len(temp) / 300)
responseNumber = 1
counter = 0

for i in range(len(temp)):
    if responseNumber == 300:
        counter += 1
        print("Call no. " + str(counter) + " of " + str(total_calls))
        responseNumber = 0
    user = temp[i]
    query = {"_id":_id}
    test = sFollowing.find_one(query)
    if type(test) != dict:
        try:
            responseNumber += 1
            response = api.get_user(id=user, user_fields=fields)
            userDoc = {
                '_id':_id,
                'username':response.data.username,
                'userID':response.data.id,
                'description':response.data.description,
                'location':response.data.location,
                'followers':response.data.public_metrics['followers_count'],
                'following':response.data.public_metrics['following_count'],
                'tweet_count':response.data.public_metrics['tweet_count'],
                'verified':response.data.verified,
                'profile_image_url':response.data.profile_image_url
            }

            sFollowing.insert_one(userDoc)
        except:
            print(str(user) + " no longer exists")
            df_sFollowingEdges = df_sFollowingEdges.loc[df_sFollowingEdges['target'] != temp[i]]
    else:
        pass
    _id += 1

df_sFollowingEdges.to_csv('sFollowing/test.csv',index=False)

In [7]:
# Add geocoded Data to the database
query = {'location': {'$ne': None}}
geoData = pd.DataFrame(list(sFollowing.find(query).sort("_id",1)))

geoData['lat'] = geoData['location'].apply(mongosna.geocode_lat)
geoData['lng'] = geoData['location'].apply(mongosna.geocode_lng)

lat = list(geoData['lat'])
lng = list(geoData['lng'])
geo_ids = list(geoData['_id'])
iter = 0

for id in geo_ids:
    query = { '_id' : id }
    value = { '$set' : { 'lat' : lat[iter], 'lng': lng[iter]}}
    iter += 1
    sFollowing.update_one(query,value)

In [ ]:
# Check if user is in montana
query = {"location": {"$regex": ".*mt.*|.*missoula.*|.*last best place.*|.*big sky.*","$options":"i"}}
mt_users = pd.DataFrame(list(sFollowing.find(query)))
mt_users['description'] = mt_users.replace({r'\s+$': '', r'^\s+': ''}, regex=True, inplace=True)

# Add 'in_montana' column to DataFrame
df_sFollowing.drop(columns="in_montana",inplace=True)
mt_ids = list(mt_users['userID'])
user_id = list(df_sFollowing['userID'])
bool_list = []

for id in user_id:
    if id in mt_ids:
        bool_list.append(True)
    else:
        bool_list.append(False)
df_sFollowing.insert(loc=10,column='in_montana',value=bool_list)

# Add 'in_montana' to database

in_montana = list(df_sFollowing['in_montana'])
iter = 0
for id in user_id:
    query = { 'userID' : id }
    value = { '$set' : { 'in_montana' : in_montana[iter]}}
    iter += 1
    sFollowing.update_one(query,value)


## Combined Followers and Following
### Exporting Database Aggregation
We aggregate the data by matching:
* `in_montana` as `true`,
* `modularity_class` individually from 0 to 6<br>
and then sorting the matches by "betweenesscentrality", "eigencentrality" and "closnesscentrality", which was obtained by analyzing the users using Gephi. We limit the output to 75 users and include just their username, centrality value and modularity class. Once we have obtained the top 75 users based on centrality, we search for users that are contained in all groups, inferring from the output that if a user is in all 3 groups, that they would be the most influential of that modularity class.

In [83]:
keywords = ["betweenesscentrality","closnesscentrality","eigencentrality"]

for i in range(len(keywords)):
    aggregation = [
        { "$match": {'in_montana': True}},
        { "$sort": {'{}'.format(keywords[i]): -1}},
        { "$limit": 75},
        { "$project": { "_id":0, "Label":1, "{}".format(keywords[i]):1, "modularity_class": 1}}
    ]
    test = pd.DataFrame(list(combined.aggregate(aggregation)))
    test.to_csv("combined/{}.csv".format(keywords[i]),index=False)
    for t in range(7):
        aggregation = [
            { "$match": {"$and": [{'in_montana': True},{"modularity_class": t}]}},
            { "$sort": {'{}'.format(keywords[i]): -1}},
            { "$limit": 75},
            { "$project": { "Label": 1, "{}".format(keywords[i]) : 1, "_id": 0, "modularity_class": 1}}
        ]
        test = pd.DataFrame(list(combined.aggregate(aggregation)))
        test.to_csv("combined/modClass{}/{}.csv".format(t,keywords[i]),index=False)

In [84]:
bc_combined = pd.read_csv("combined/betweenesscentrality.csv")
cc_combined = pd.read_csv("combined/closnesscentrality.csv")
ec_combined = pd.read_csv("combined/eigencentrality.csv")

influencer_combined = list(set(bc_combined["Label"]) & set(cc_combined["Label"]) & set(ec_combined["Label"]))
influencers = pd.Series(influencer_combined)
influencers.to_csv("combined/influencers.csv",index=False)

for i in range(7):
    bc_df = pd.read_csv("combined/modClass{}/betweenesscentrality.csv".format(i))
    cc_df = pd.read_csv("combined/modClass{}/closnesscentrality.csv".format(i))
    ec_df = pd.read_csv("combined/modClass{}/eigencentrality.csv".format(i))

    influencers = list(set(bc_df["Label"]) & set(cc_df["Label"]) & set(ec_df["Label"]))
    temp = pd.Series(influencers)
    temp.to_csv("combined/modClass{}/influencers.csv".format(i),index=False)